Below is the max flow formulation from the example. We need to alter the network such that solving max flow over gives us a solution to the problem stated in the exercise.

Namely, the total number of people that can enter/exit the following countries are:

Country 2: 550

Country 3: 500

In the markdown cell below, describe how you are modifying the network

If you cannot figure out how to modify the network accordingly, you may solve the problem by modifying the LP formulation instead.

Modification:

In [8]:
using Cbc, JuMP, NamedArrays

#################################################
# Make alterations to the sets/parameters below #
#################################################

# Sets
C = [:S,1,2,3,4,5,:T]
A = [(:S, 1),(:S, 2),(1, 2),(2, 1),(1, 3),(3, 1),(2, 3),(3, 2),(3, 4),(4, 3),(2, 4),(4, 2),(4, 5),(5,4),(2, 5),(5, 2),(2, :T),(4, :T),(5, :T)]

# Parameters
q = Dict(zip(A, [500, 400, 200, 230, 210, 400, 180, 270, 600, 200, 410, 200, 340, 210, 520, 480, 260, 180, 420]))

################################################
# general model, should not need to be changed #
################################################

m = Model()

@variable(m, 0 <= f[union(A,[(:T, :S)])])

@objective(m, Max, f[(:T, :S)])

@constraint(m, flowconserve[i in C], sum(f[(j, i)] for j in C if (j, i) in union(A,[(:T, :S)])) == sum(f[(i, j)] for j in C if (i, j) in union(A,[(:T, :S)])))
@constraint(m, capacity[a in A], f[a] <= q[a])
@constraint(m, maxinput_2, f[(:S,2)] <= 550)
@constraint(m, maxoutput_2, f[(2,1)] +  f[(2,3)] + f[(2,4)] + f[(2,5)] + + f[(2,:T)] <= 550)
@constraint(m, maxinput_3, f[(1,3)] + f[(2,3)] + f[(4,3)] <= 500)
@constraint(m, maxoutput_3, f[(3,1)] +  f[(3,2)] + f[(3,4)] <= 500)

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Presolve 9 (-21) rows, 19 (-1) columns and 49 (-22) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 4.7050271e-05 ( 0.0047050271%) - largest zero change 4.6564243e-05
0  Obj -0 Dual inf 1.9999216 (2)
7  Obj 759.9092
Optimal - objective value 760
After Postsolve, objective 760, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 760 - 7 iterations time 0.002, Presolve 0.00


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 760.0
  Objective bound      : 760.0
  Primal solution :
    f[(1, 2)] : 150.0
    f[(1, 3)] : 210.0
    f[(2, 1)] : 0.0
    f[(2, 3)] : 0.0
    f[(2, 4)] : 0.0
    f[(2, 5)] : 290.0
    f[(2, :T)] : 260.0
    f[(3, 1)] : 0.0
    f[(3, 2)] : 0.0
    f[(3, 4)] : 210.0
    f[(4, 2)] : 0.0
    f[(4, 3)] : 0.0
    f[(4, 5)] : 30.0
    f[(4, :T)] : 180.0
    f[(5, 2)] : 0.0
    f[(5, 4)] : 0.0
    f[(5, :T)] : 320.0
    f[(:S, 1)] : 360.0
    f[(:S, 2)] : 400.0
    f[(:T, :S)] : 760.0

* Work counters
  Solve time (sec)   : 0